In [2]:
from scipy.fft import fft, ifft, fftshift, fft2, ifft2
from scipy.signal import stft
from scipy.io.wavfile import read as wave_read

import numpy as np
from skimage import io
import pywt # note, you'll need to install via "pip install PyWavelets" in either a conda shell or a Colab notebook

import matplotlib.pyplot as plt
%matplotlib inline 

**Problem 1**: Generate a short-time Fourier transform decomposition of Cantina60.wav.  Explain your parameter choices, and show possible alternative representations of your STFT and why you finally chose the one that you did.  You should also make recourse to the song itself and use that to help you explain your decision making process.   

**Problem 2**: Using beans.jpg, or whichever (school appropriate) image you prefer, and following the model for how we built an image approximation using the approximation coefficiets from a wavelet decomposition, create corresponding images for each of the different levels of detail in decomposition.  Use three levels of decomposition.  

**Problem 3**: Using the results from above, generate Fourier transforms of each detail image you reconstruct.  Explain and compare your results from doing this.  